In [29]:
from __future__ import print_function, division
from builtins import range

In [30]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit as sigmoid
from sklearn.utils import shuffle
from datetime import datetime

In [31]:
from scipy.spatial.distance import cosine as cos_dist
from sklearn.metrics.pairwise import pairwise_distances

In [32]:
from glob import glob

In [33]:
import os
import sys
import string

sys.path.append(os.path.abspath('..'))
from rnn_class.brown import get_sentences_with_word2idx_limit_vocab as get_brown

In [34]:
def remove_punctuation_2(s):
    return s.translate(None, string.punctuation)

def remove_punctuation_3(s):
    return s.translate(str.maketrans('','',string.punctuation))

if sys.version.startswith('2'):
    remove_punctuation = remove_punctuation_2
else:
    remove_punctuation = remove_punctuation_3

In [35]:
def get_wiki():
  V = 2000
  files = glob('../large_files/descricao_vagas.txt')
  all_word_counts = {}
  for f in files:
    for line in open(f):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          for word in s:
            if word not in all_word_counts:
              all_word_counts[word] = 0
            all_word_counts[word] += 1
  print("finished counting")

  V = min(V, len(all_word_counts))
  all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)

  top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
  word2idx = {w:i for i, w in enumerate(top_words)}
  unk = word2idx['<UNK>']

  sents = []
  for f in files:
    for line in open(f):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:

          sent = [word2idx[w] if w in word2idx else unk for w in s]
          sents.append(sent)
  return sents, word2idx

In [36]:
print(get_wiki)

<function get_wiki at 0x0000018F6690A550>


In [37]:
def train_model(savedir):

  sentences, word2idx = get_wiki() #get_brown()



  vocab_size = len(word2idx)



  window_size = 5
  learning_rate = 0.025
  final_learning_rate = 0.0001
  num_negatives = 5 
  epochs = 20
  D = 50 


 
  learning_rate_delta = (learning_rate - final_learning_rate) / epochs


 
  W = np.random.randn(vocab_size, D) 
  V = np.random.randn(D, vocab_size) 


 
  p_neg = get_negative_sampling_distribution(sentences, vocab_size)


  
  costs = []


  
  total_words = sum(len(sentence) for sentence in sentences)
  print("total number of words in corpus:", total_words)

  
  threshold = 1e-5
  p_drop = 1 - np.sqrt(threshold / p_neg)


 
  for epoch in range(epochs):
    
    np.random.shuffle(sentences)

   
    cost = 0
    counter = 0
    t0 = datetime.now()
    for sentence in sentences:
     
      sentence = [w for w in sentence \
        if np.random.random() < (1 - p_drop[w])
      ]
      if len(sentence) < 2:
        continue


     
      randomly_ordered_positions = np.random.choice(
        len(sentence),
        size=len(sentence),#np.random.randint(1, len(sentence) + 1),
        replace=False,
      )

      
      for pos in randomly_ordered_positions:
       
        word = sentence[pos]

       
        context_words = get_context(pos, sentence, window_size)
        neg_word = np.random.choice(vocab_size, p=p_neg)
        targets = np.array(context_words)

        
        c = sgd(word, targets, 1, learning_rate, W, V)
        cost += c
        c = sgd(neg_word, targets, 0, learning_rate, W, V)
        cost += c

      counter += 1
      if counter % 100 == 0:
        sys.stdout.write("processed %s / %s\r" % (counter, len(sentences)))
        sys.stdout.flush()
        # break


    
    dt = datetime.now() - t0
    print("epoch complete:", epoch, "cost:", cost, "dt:", dt)

    
    costs.append(cost)

   
    learning_rate -= learning_rate_delta


  
  plt.plot(costs)
  plt.show()


 
  if not os.path.exists(savedir):
    os.mkdir(savedir)

  with open('%s/word2idx.json' % savedir, 'w') as f:
    json.dump(word2idx, f)

  np.savez('%s/weights.npz' % savedir, W, V)

  
  return word2idx, W, V


In [38]:
def get_negative_sampling_distribution(sentences, vocab_size):
  # Pn(w) = prob of word occuring
  # we would like to sample the negative samples
  # such that words that occur more often
  # should be sampled more often

  word_freq = np.zeros(vocab_size)
  #word_count = sum(len(sentence) for sentence in sentences)
  for sentence in sentences:
      for word in sentence:
          word_freq[word] += 1
        
  zero_count_idx = [i for i in range(len(word_freq)) if (word_freq[i]==0)]
  print("zero_cound_idx", zero_count_idx)
  #sys.exit()

  # smooth it
  p_neg = word_freq**0.75

  # normalize it
  p_neg = p_neg / p_neg.sum()

  assert(np.all(p_neg > 0))
  return p_neg

In [39]:
def get_context(pos, sentence, window_size):
 

  start = max(0, pos - window_size)
  end_  = min(len(sentence), pos + window_size)

  context = []
  for ctx_pos, ctx_word_idx in enumerate(sentence[start:end_], start=start):
    if ctx_pos != pos:
      
      context.append(ctx_word_idx)
  return context


def sgd(input_, targets, label, learning_rate, W, V):
  
  activation = W[input_].dot(V[:,targets])
  prob = sigmoid(activation)

 
  gV = np.outer(W[input_], prob - label) # D x N
  gW = np.sum((prob - label)*V[:,targets], axis=1) # D

  V[:,targets] -= learning_rate*gV # D x N
  W[input_] -= learning_rate*gW # D

  
  cost = label * np.log(prob + 1e-10) + (1 - label) * np.log(1 - prob + 1e-10)
  return cost.sum()

In [40]:
def load_model(savedir):
  with open('%s/word2idx.json' % savedir) as f:
    word2idx = json.load(f)
  npz = np.load('%s/weights.npz' % savedir)
  W = npz['arr_0']
  V = npz['arr_1']
  return word2idx, W, V

In [41]:
def analogy(pos1, neg1, pos2, neg2, word2idx, idx2word, W):
  V, D = W.shape

  
  print("testing: %s - %s = %s - %s" % (pos1, neg1, pos2, neg2))
  for w in (pos1, neg1, pos2, neg2):
    if w not in word2idx:
      print("Sorry, %s not in word2idx" % w)
      return

  p1 = W[word2idx[pos1]]
  n1 = W[word2idx[neg1]]
  p2 = W[word2idx[pos2]]
  n2 = W[word2idx[neg2]]

  vec = p1 - n1 + n2

  distances = pairwise_distances(vec.reshape(1, D), W, metric='cosine').reshape(V)
  idx = distances.argsort()[:10]

 
  best_idx = -1
  keep_out = [word2idx[w] for w in (pos1, neg1, neg2)]
 
  for i in idx:
    if i not in keep_out:
      best_idx = i
      break
  

  print("got: %s - %s = %s - %s" % (pos1, neg1, idx2word[best_idx], neg2))
  print("closest 10:")
  for i in idx:
    print(idx2word[i], distances[i])

  print("dist to %s:" % pos2, cos_dist(p2, vec))

In [42]:
def test_model(word2idx, W, V):
 

  idx2word = {i:w for w, i in word2idx.items()}

  for We in (W, (W + V.T) / 2):
    print("**********")

    analogy('machine', 'leraning', 'big', 'data', word2idx, idx2word, We)

In [43]:
word2idx, W, V = train_model('w2v_model')
test_model(word2idx, W, V)

finished counting
zero_cound_idx [0]


C:\Users\lucas.miranda\AppData\Local\Temp\ipykernel_11120\2352422927.py:21: RuntimeWarning: invalid value encountered in true_divide
  p_neg = p_neg / p_neg.sum()


AssertionError: 